In [172]:
# import libraries
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.animation import FuncAnimation
import mesa
from mesa import Agent, Model
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector
import numpy as np
import math
from IPython.display import HTML
from matplotlib import rc
import pandas as pd
import numpy.lib.recfunctions as rf
import time


This code runs with mesa version 3.00 or higher

In [173]:
# check mesa version
print("Mesa version: ", mesa.__version__)

Mesa version:  3.1.0


In [174]:
# Function to calculate Euclidean distance (distance between two points)
def euclidean_distance(pos1, pos2):
    # Use the formula to calculate distance between two coordinates
    return math.sqrt((pos1[0] - pos2[0]) ** 2 + (pos1[1] - pos2[1]) ** 2)

# Function to calculate integer grid points between two coordinates
def connect2(ends):
    d0, d1 = np.diff(ends, axis=0)[0]
    if np.abs(d0) > np.abs(d1): 
        return np.c_[np.arange(ends[0, 0], ends[1,0] + np.sign(d0), np.sign(d0), dtype=np.int32),
                     np.arange(ends[0, 1] * np.abs(d0) + np.abs(d0)//2,
                               ends[0, 1] * np.abs(d0) + np.abs(d0)//2 + (np.abs(d0)+1) * d1, d1, dtype=np.int32) // np.abs(d0)]
    else:
        return np.c_[np.arange(ends[0, 0] * np.abs(d1) + np.abs(d1)//2,
                               ends[0, 0] * np.abs(d1) + np.abs(d1)//2 + (np.abs(d1)+1) * d0, d0, dtype=np.int32) // np.abs(d1),
                     np.arange(ends[0, 1], ends[1,1] + np.sign(d1), np.sign(d1), dtype=np.int32)]

# Transforms the raw model grid data into usable arrays for the unblocked vision function
def prep_vision(vision_area, obstacles):
    # Find all obstacles within the vision range
    vision_array = np.asarray(vision_area)
    obstacles = np.asarray(obstacles)
    nrows, ncols = vision_array.shape
    dtype={'names':['f{}'.format(i) for i in range(ncols)], 'formats':ncols * [vision_array.dtype]}
    obstacles_in_sight = np.intersect1d(vision_array.view(dtype), obstacles.view(dtype))
    obstacles_in_sight = rf.structured_to_unstructured(obstacles_in_sight).tolist()
    
    # Trim vision_array down to only the edges (optimisation)
    xtrim = np.array([np.min(vision_array[:, 0]), np.max(vision_array[:, 0])])
    ytrim = np.array([np.min(vision_array[:, 1]), np.max(vision_array[:, 1])])
    
    xmask = np.isin(element = vision_array[:, 0], test_elements=xtrim)
    ymask = np.isin(element = vision_array[:, 1], test_elements=ytrim)
    
    xarray = vision_array[xmask]
    yarray = vision_array[ymask]
    vision_array = np.vstack((xarray, yarray))
    
    return vision_array, obstacles_in_sight

# Function to find all visible tiles to an agent around obstacles; call this function for enhanced vision behaviour
def unblocked_vision(pos, vision_area, obstacles):
    # Prepare data
    vision_array, obstacles_in_sight = prep_vision(vision_area, obstacles)
    x0 = pos[0]
    y0 = pos[1]
    vis_area = []
    
    # Iterate over all the tiles within the vision area
    for tile in vision_array[:, 0:2]:
        x1 = tile[0]
        y1 = tile[1]
        # Calculate simple sightline for straight lines (2D array); not covered by function
        if x0 == x1:
            if y0 > y1:
                liny = np.vstack(np.arange(y1, y0 + 1)[::-1])
            else:
                liny = np.vstack(np.arange(y0, y1 + 1))
            linx = (np.ones((len(liny[:, 0]), 1))*x0).astype(int)
            visline = np.hstack((linx, liny))
        elif y0 == y1:
            if x0 > x1:
                linx = np.vstack(np.arange(x1, x0 + 1)[::-1])
            else:
                linx = np.vstack(np.arange(x0, x1 + 1))
            liny = (np.ones((len(linx[:, 0]), 1))*y0).astype(int)
            visline = np.hstack((linx, liny))
            
        # Use function to obtain sightline for diagonals (2D array)
        else:
            coords = np.array([[x0, y0],
                               [x1, y1]])
            visline = connect2(coords)
        
        # Add all possible sighted tiles (not blocked by an obstacle) to new vision list of tuples
        for loc in visline[:, 0:2]:
            vis_area.append((int(loc[0]), int(loc[1])))
            if [loc[0], loc[1]] in obstacles_in_sight:
                break
                
    # Remove duplicates from the vision list
    vision_area = list(set(vis_area))
                
    return vision_area

# Tool to plot the vision area of a single agent; only use for development purposes
def plot_vision(pos, vision, obstacles, vision_area):
    viz = np.zeros((vision*2 + 1, vision*2 + 1))
    posx = pos[0] - vision
    posy = pos[1] - vision
    for i in range(len(viz[0, :])):
        for j in range(len(viz[:, 0])):
            y = vision*2 - j
            if (i + posx, j + posy) in vision_area:
                viz[y, i] = 1
            if (i + posx, j + posy) in obstacles:
                viz[y, i] = 2
            if (i + posx, j + posy) == pos:
                viz[y, i] = 3
                
    fig, ax = plt.subplots(figsize=(4, 4))
    cmap = mcolors.ListedColormap(['black', 'green', 'white', 'red'])
    bounds = [0, 1, 2, 3]
    norm = mcolors.BoundaryNorm(bounds, cmap.N, extend='max')
    ax.imshow(viz, cmap=cmap, norm=norm)


# Base Model

In [175]:
###################
### AGENT CLASS ###
###################

# Define the NavigationAgent class
class NavigationAgent(Agent):
    def __init__(self, model, vision=5, real_vision = True): # Default vision range of 5 cells
        super().__init__(model)  # MESA `Agent` class initialization, auto-assigns unique_id in Mesa 3.0
        # Attributes of each agent
        self.found_exit = False  # Track if agent has reached the exit
        self.previous_pos = None  # Previous position of the agent
        self.real_vision = real_vision
        self.vision = vision  # Vision range of the agent

    # Function to move the agent towards the exit
    def move_towards_exit(self):
        self.previous_pos = self.pos  # Store the current position before moving
        # MESA `get_neighborhood` function retrieves nearby cells based on vision range
        possible_steps = self.model.grid.get_neighborhood(self.pos, moore=True, include_center=False)
        
        min_distance = float('inf')  # Start with a very large distance
        best_step = None  # Initialize best step as None
        
        # Check each possible step to find the one closest to the exit
        for step in possible_steps:
            # Only consider steps that don't have obstacles and have less than 8 agents
            if step not in self.model.obstacles and len(self.model.grid.get_cell_list_contents(step)) < 8:
                dist = []
                for loc in self.model.exit_location:
                    dist.append(euclidean_distance(step, loc))
                dist = np.min(dist)  # Distance to the exit
                if dist < min_distance:
                    min_distance = dist
                    best_step = step  # Update best step to be closer to the exit
        if best_step:
            # MESA `move_agent` function moves the agent to the new cell
            self.model.grid.move_agent(self, best_step)

    # Function to move the agent randomly if the exit is not in sight
    def move_randomly(self):
        self.previous_pos = self.pos
        possible_steps = self.model.grid.get_neighborhood(self.pos, moore=True, include_center=False)
        
        # Filter steps to only those without obstacles and with fewer than 8 agents
        valid_steps = [step for step in possible_steps if step not in self.model.obstacles and len(self.model.grid.get_cell_list_contents(step)) < 8]
        
        if valid_steps:
            # Randomly choose a valid position and move there
            random_step = self.random.choice(valid_steps)
            # MESA `move_agent` function moves the agent to the chosen position
            self.model.grid.move_agent(self, random_step)

    # Define the actions the agent will take in each step
    # TODO: Add behaviour of signs
    def step(self):
        # If the agent is at the exit, mark as exited
        if self.pos in self.model.exit_location:
            self.found_exit = True  # Set the agent's exit status to True
            self.model.grid.remove_agent(self)  # MESA function to remove the agent from the grid
            self.remove()  #self.remove() to remove from AgentSet
            self.model.cumulative_exited += 1  # Count this agent in cumulative exited agents
        else:
            # MESA `get_neighborhood` checks the agent's vision area for the exit
            vision_area = self.model.grid.get_neighborhood(self.pos, moore=True, radius=self.vision, include_center=False)
            
            # Obtain all obstacles within the vision area (2D array)
            if self.real_vision:
                vision_area = unblocked_vision(self.pos, vision_area, self.model.obstacles)

            # Tool for visualisation of vision area 
            # ONLY USE WITH 1 AGENT AND 1 TIMESTEP
            # ------------------------------------ START -----------------------------------
            
            check_vis = False
            if check_vis:
                plot_vision(self.pos, self.vision, self.model.obstacles, vision_area)
            
            # ------------------------------------ END -----------------------------------
            
            exits = []
            for loc in self.model.exit_location:
                if loc in vision_area:
                    exits.append(loc)
            exit_in_vision = exits
            
            # Move towards the exit if it's in sight, otherwise move randomly
            if exit_in_vision:
                self.move_towards_exit()
            else:
                self.move_randomly()

###################
### MODEL CLASS ###
###################

# Define the model class to handle the overall environment
class FloorPlanModel(Model):
    def __init__(self, width, height, num_agents, agent_vision, agent_real_vision):
        super().__init__()  # `Model` class initialization
        
        # Basic model settings
        self.num_agents = num_agents
        self.agent_vision = agent_vision
        self.agent_real_vision = agent_real_vision
        self.grid = MultiGrid(width, height, False)  # MESA grid with dimensions; False means no wrapping
        # self.exit_location = (width - 1, height - 1)  # Place exit at the bottom-right corner
        
        # Define obstacles and signs in the grid
        floor0 = pd.read_csv("Floor0.csv", header=None)
        floor0 = pd.DataFrame.to_numpy(floor0)
        floor0 = np.flipud(floor0)
        
        GreenTiles = []
        BlackTiles = []
        WhiteTiles = []
        BlueTiles = []
        PurpleTiles = []
        YellowTiles = []
        OrangeTiles = []
        RedTiles = []
        
        for i in range(np.shape(floor0)[0]):
            for j in range(np.shape(floor0)[1]):
                if floor0[i, j] == 0:
                    GreenTiles.append((j, i))
                if floor0[i, j] == 1:
                    BlackTiles.append((j, i))
                if floor0[i, j] == 2:
                    WhiteTiles.append((j, i))
                if floor0[i, j] == 3:
                    BlueTiles.append((j, i))
                if floor0[i, j] == 4:
                    PurpleTiles.append((j, i))
                if floor0[i, j] == 5:
                    YellowTiles.append((j, i))
                if floor0[i, j] == 6:
                    OrangeTiles.append((j, i))
                if floor0[i, j] == 7:
                    RedTiles.append((j, i))

        self.outdoors = GreenTiles
        self.obstacles = BlackTiles
        self.indoors = WhiteTiles + BlueTiles
        self.exit_location = PurpleTiles
        self.stairs = YellowTiles
        self.lifts = OrangeTiles
        self.signs = RedTiles
        
        # Initialize cumulative exited count
        self.cumulative_exited = 0

        # Initialize DataCollector (MESA tool for tracking metrics across steps)
        self.datacollector = DataCollector(
            model_reporters={
                "Active Agents": lambda m: len(m.agents),  # Count of agents still active
                "Exited Agents": lambda m: sum(1 for agent in m.agents if agent.found_exit == True),
                "Cumulative Exited Agents": lambda m: m.cumulative_exited,  # Cumulative exited count
                "Agents per Cell": self.count_agents_per_cell  # Counts agents in each cell
            },
            agent_reporters={
                "Found Exit": lambda a: a.found_exit if isinstance(a, NavigationAgent) else None  # Reports exit status per agent
            }
        )

        self.place_agents(agent_vision)  # Place agents on the grid
        self.datacollector.collect(self) # Collect data at the start of the simulation

    # Function to randomly place agents in the grid
    # TODO: Change spawn rules to for every tile type
    def place_agents(self, agent_vision):
        for i in range(self.num_agents):
            agent = NavigationAgent(self, vision=agent_vision, real_vision=self.agent_real_vision)
            placed = False  # Track if the agent is successfully placed
            while not placed:
                x = self.random.randrange(self.grid.width)
                y = self.random.randrange(self.grid.height)
                cell_contents = self.grid.get_cell_list_contents((x, y))

                # Only place agent if cell has no obstacles and fewer than 8 agents
                if (x, y) not in self.obstacles and len(cell_contents) < 8 and (x, y) not in self.outdoors:
                    self.grid.place_agent(agent, (x, y))  # MESA function to place agent in grid
                    placed = True  # Mark as placed
                    # if you want to see the initial placement of the agents, uncomment the line below. This will print the initial position of the agents
                    # print(f"Agent {i} placed at: ({x}, {y})")

    # Function to count agents in each cell
    def count_agents_per_cell(self):
        agent_counts = {}  # Dictionary to store agent counts by cell position
        # MESA `coord_iter` function iterates over grid cells and their contents
        for cell in self.grid.coord_iter():
            cell_contents, (x, y) = cell  # Unpack cell contents and coordinates
            # Count NavigationAgents in each cell
            nav_agent_count = sum(1 for obj in cell_contents if isinstance(obj, NavigationAgent))
            if nav_agent_count > 0:
                agent_counts[(x, y)] = nav_agent_count
        return agent_counts
    
    # Function to get the grid data for visualization
    def get_grid(self):
        # 0: outdoors, 1: obstacle, 2/3: indoors, 4: exit, 7: sign, 8: agent
        grid_data = np.zeros((self.grid.width, self.grid.height))
        
        # Mark obstacles on the grid
        for x, y in self.obstacles:
            grid_data[y, x] = 1

        for x, y in self.indoors:
            grid_data[y, x] = 2

        # Mark special areas on the grid
        for x, y in self.stairs:
            grid_data[y, x] = 5

        for x, y in self.lifts:
            grid_data[y, x] = 6

        # Mark agents on the grid
        for agent in self.agents:
            if isinstance(agent, NavigationAgent):
                x, y = agent.pos
                grid_data[y, x] = 8
        
        # Mark signs and exit
        for x, y in self.exit_location:
            grid_data[y, x] = 4

        for x, y in self.signs:
            grid_data[y, x] = 7

        return grid_data

    # Model step function to update the simulation
    def step(self):
        self.agents.do("step")  # MESA 3.0 function to execute the `step` function of each agent
        self.datacollector.collect(self)  # MESA DataCollector collects metrics at each step

    

In [177]:
# Run the model and see the results
model = FloorPlanModel(127, 127, 100, 10, True) # run the model with 30x30 grid and 100 agents and a vision of 5
for i in range(100): # run the model for 100 steps
    model.step() # step the model by 1

# Collect the data from the model
model_data = model.datacollector.get_model_vars_dataframe()
agent_data = model.datacollector.get_agent_vars_dataframe()

In [178]:
model_data # print the model data

,Active Agents,Exited Agents,Cumulative Exited Agents,Agents per Cell
0,100,0,0,"{(2, 81): 1, (11, 80): 1, (11, 81): 1, (20, 86..."
1,100,0,0,"{(1, 82): 1, (10, 79): 1, (10, 80): 1, (19, 85..."
2,100,0,0,"{(1, 83): 1, (9, 79): 2, (18, 84): 1, (19, 62)..."
3,99,0,1,"{(0, 84): 1, (8, 79): 2, (17, 83): 1, (19, 86)..."
4,97,0,3,"{(7, 78): 2, (16, 82): 1, (18, 85): 1, (20, 73..."
...,...,...,...,...
96,61,0,39,"{(22, 39): 1, (24, 69): 1, (26, 76): 1, (28, 4..."
97,61,0,39,"{(21, 38): 1, (23, 68): 1, (27, 77): 1, (29, 3..."
98,61,0,39,"{(20, 37): 1, (22, 69): 1, (28, 57): 1, (28, 7..."
99,61,0,39,"{(20, 36): 1, (23, 70): 1, (27, 58): 1, (29, 4..."


In [179]:
agent_data # print the agent data

Found Exit
Step AgentID            
0    1             False
     2             False
     3             False
     4             False
     5             False
...                  ...
100  93            False
     94            False
     98            False
     99            False
     100           False

[7293 rows x 1 columns]

# Visualization with User Interface
You don't have to understand every single line in the visualisation code below. Please understand the code to extend that you can introduce changes to it when needed

In [180]:
# Visualization Function
def plot_grid(model, ax):
    rc("animation", embed_limit=4096)  # Set a higher limit in MB to allow smoother animation playback
    grid_data = model.get_grid()  # Retrieve the current state of the grid from the model
    ax.clear()  # Clear any previous plots on the axes to prevent overlap in visualizations

    # Define color mappings:
    # 0 (outdoors) -> green, 1 (obstacle) -> black, 2 (indoors) -> white,
    # 3 (door) -> white, 4 (exit) -> purple/mediumorchid, 5 (yellow) -> stairs, 6 (lifts) -> orange, 7 (signs) -> red, 8 (agent) -> blue
    cmap = mcolors.ListedColormap(['green', 'black', 'white', 'white', 'mediumorchid', 'yellow', 'orange', 'red', 'blue'])
    bounds = [0, 1, 2, 3, 4, 5, 6, 7, 8]  # Boundaries to separate each category
    norm = mcolors.BoundaryNorm(bounds, cmap.N, extend='max')  # Normalizes values to assign colors to each category
    
    # Display the grid data with color mapping applied.
    # Setting 'origin' to 'lower' places the (0,0) coordinate at the bottom-left.
    ax.imshow(grid_data, cmap=cmap, norm=norm, origin='lower')
    
    # Customize grid display: set grid to start from -0.5 with labels at intervals of 1
    # Set minor ticks for cell boundaries, with thicker and darker lines
    ax.set_xticks(np.arange(-0.5, model.grid.width, 1), minor=True)
    ax.set_yticks(np.arange(-0.5, model.grid.height, 1), minor=True)
    ax.grid(which='minor', color='black', linestyle='-', linewidth=0.3)  # Thicker, darker lines for cell boundaries

    # Set major ticks for labels at intervals of 5, with lighter and thinner lines
    ax.set_xticks(np.arange(0, model.grid.width, 5), minor=False)
    ax.set_yticks(np.arange(0, model.grid.height, 5), minor=False)
    ax.grid(which='major', color='lightgray', linestyle='-', linewidth=0)  # invisible line

    
    # Label the exit point on the grid in red text for easy identification
    #exit_x, exit_y = model.exit_location
    #ax.text(exit_x, exit_y, 'EXIT', ha='center', va='center', fontsize=12, color='red', fontweight='bold')

    # Add labels for each sign location in black text for visibility
    #for sign_pos in model.signs:
        #x, y = sign_pos
        #ax.text(x, y, 'SIGN', ha='center', va='center', fontsize=10, color='black', fontweight='bold')

    # Set the title to show the current step number in the model
    ax.set_title(f"Step {model.steps}", fontsize=16)
    ax.tick_params(axis='both', which='major', labelsize=7)  # Control tick label size
    
    # Draw movement arrows for agents to show the direction they are traveling
    for agent in model.agents:
        # Only draw arrow if the agent has moved and is a NavigationAgent
        if isinstance(agent, NavigationAgent) and agent.previous_pos:
            start_x, start_y = agent.previous_pos  # Previous position of the agent
            end_x, end_y = agent.pos  # Current position of the agent
            
            # Draw an arrow from the previous position to the current position
            ax.arrow(
                start_x, start_y,
                end_x - start_x, end_y - start_y,
                head_width=0.3, head_length=0.3, fc='yellow', ec='yellow'  # Yellow arrow for movement direction
            )

# Animation Update Function
def update(frame, model, ax):
    # For every frame, update the model state if it's not the first frame
    if frame > 0:
        model.step()  # Run one step of the model simulation
    plot_grid(model, ax)  # Redraw the grid with updated agent positions
    
# Run the Animation with a larger figure size
def run_animation(model, steps):
    fig, ax = plt.subplots(figsize=(10, 10))  # Create a 10x10 figure for the plot
    fig.tight_layout()
    plot_grid(model, ax)  # Plot the initial grid state
    # Create an animation that updates the grid for each step
    anim = FuncAnimation(fig, update, frames=steps+1, fargs=(model, ax), repeat=False)
    plt.close(fig)  # Close the figure after animation creation to avoid duplicate displays
    return anim

# Your imports and function definitions remain the same, up until where you initialize and run the model
import ipywidgets as widgets  # For interactive widgets (slider, button)
from IPython.display import display, HTML  # To display widgets and HTML animations
import time  # For tracking elapsed time

# Slider to choose the number of agents
agent_slider = widgets.IntSlider(
    value=10,      # Default starting number of agents
    min=10,        # Minimum number of agents allowed
    max=500,       # Maximum number of agents allowed
    step=10,       # Step size for slider increments
    description='Num Agents:',  # Label for the slider
    continuous_update=False     # Only update value when slider is released
)

# Slider to choose the number of time steps (frames) for the animation
time_step_slider = widgets.IntSlider(
    value=50,      # Default starting number of steps
    min=1,         # Minimum time steps allowed
    max=600,       # Maximum time steps allowed
    step=1,        # Step size for slider increments
    description='Time Steps:',  # Label for the slider
    continuous_update=False     # Only update value when slider is released
)

# Slider to choose the vision range (vision radius) of agents
vision_slider = widgets.IntSlider(
    value=5,      # Default starting vision range
    min=1,        # Minimum vision range
    max=20,       # Maximum vision range
    step=1,       # Step size for slider increments
    description='Vision:',  # Label for the slider
    continuous_update=False     # Only update value when slider is released
)

# Button to start the simulation with current slider settings
run_button = widgets.Button(description="Run Simulation")

# Output widget for displaying the animation and elapsed time
output_widget = widgets.Output()

# Label to show elapsed time during the simulation
elapsed_time_label = widgets.Label(value="Elapsed time: 0.0 seconds")

# Flag variable to control timing function
stop_timer = False

# Display the interface with sliders, button, label, and output area
display(agent_slider, time_step_slider, vision_slider, run_button, elapsed_time_label, output_widget)

# Define the function to initialize and run the model
def run_model(change):
    global stop_timer, model_data, agent_data  # Allow variables to be accessed globally
    with output_widget:  # Use output widget to display the animation
        output_widget.clear_output()  # Clear any previous output
        num_agents = agent_slider.value  # Get the number of agents from the slider
        time_steps = time_step_slider.value  # Get the number of steps from the slider
        agent_vision = vision_slider.value  # Get the vision range from the slider
        model = FloorPlanModel(width=127, height=127, num_agents=num_agents, agent_vision=agent_vision, agent_real_vision=True)  # Initialize the model


        # Reset timer flag and start timing for the animation
        stop_timer = False
        start_time = time.time()

        def update_time_label():
            while not stop_timer:  # Keep updating time label until timer stops
                elapsed_time = time.time() - start_time
                elapsed_time_label.value = f"Elapsed time: {elapsed_time:.1f} seconds"
                time.sleep(0.1)  # Update every 0.1 seconds for real-time effect

        # Start elapsed time tracking in a separate thread
        import threading
        timer_thread = threading.Thread(target=update_time_label, daemon=True)
        timer_thread.start()

        # Run the animation with selected number of steps
        anim = run_animation(model, steps=time_steps)

        # Display the animation output in HTML format
        output = HTML(anim.to_jshtml())
        display(output)

        # Stop the timer after the simulation completes
        stop_timer = True
        elapsed_time = time.time() - start_time
        elapsed_time_label.value = f"Total elapsed time: {elapsed_time:.1f} seconds"

        # Retrieve and display model and agent data after simulation completes
        model_data = model.datacollector.get_model_vars_dataframe()  # Data for the model over time
        agent_data = model.datacollector.get_agent_vars_dataframe()  # Data for each agent over time
    return model, model_data, agent_data  # Return model and data for further inspection

# Attach the run_model function to the run button click event
run_button.on_click(run_model)

IntSlider(value=10, continuous_update=False, description='Num Agents:', max=500, min=10, step=10)

IntSlider(value=50, continuous_update=False, description='Time Steps:', max=600, min=1)

IntSlider(value=5, continuous_update=False, description='Vision:', max=20, min=1)

Button(description='Run Simulation', style=ButtonStyle())

Label(value='Elapsed time: 0.0 seconds')

Output()

In [181]:
print("\nModel Data:")
model_data


Model Data:


,Active Agents,Exited Agents,Cumulative Exited Agents,Agents per Cell
0,200,0,0,"{(3, 89): 1, (4, 80): 1, (6, 80): 1, (10, 82):..."
1,198,0,2,"{(2, 88): 1, (5, 79): 1, (6, 79): 1, (9, 81): ..."
2,196,0,4,"{(1, 87): 1, (6, 78): 2, (8, 80): 1, (9, 79): ..."
3,192,0,8,"{(1, 86): 1, (7, 79): 1, (8, 79): 1, (8, 84): ..."
4,190,0,10,"{(1, 85): 1, (6, 78): 1, (7, 78): 1, (7, 83): ..."
...,...,...,...,...
196,97,0,103,"{(20, 38): 1, (20, 41): 1, (21, 40): 1, (21, 6..."
197,97,0,103,"{(20, 40): 1, (20, 62): 1, (21, 39): 1, (22, 3..."
198,97,0,103,"{(19, 62): 1, (20, 38): 1, (20, 41): 1, (22, 4..."
199,96,0,104,"{(20, 39): 1, (21, 42): 1, (23, 39): 1, (23, 4..."


In [247]:
print("\nAgent Data:")
agent_data.head(40)



Agent Data:


Found Exit
Step AgentID            
0    1             False
     2             False
     3             False
     4             False
     5             False
     6             False
     7             False
     8             False
     9             False
     10            False
     11            False
     12            False
     13            False
     14            False
     15            False
     16            False
     17            False
     18            False
     19            False
     20            False
     21            False
     22            False
     23            False
     24            False
     25            False
     26            False
     27            False
     28            False
     29            False
     30            False
     31            False
     32            False
     33            False
     34            False
     35            False
     36            False
     37            False
     38            False
     39            False
     40            False

In [202]:
#  Uncomment the code below to display the 'Agents per Cell' data for each step

#print("Agents per Cell at each step:")
#for step, agents_per_cell in model_data["Agents per Cell"].items():
#    print(f"\nStep {step}:")
#    for cell, count in agents_per_cell.items():
#        print(f"  Cell {cell}: {count} agent(s)")